In [1]:
import pandas as pd
import numpy as np

In [2]:
on_faults = pd.read_pickle('../data/on_faults.pkl')

In [3]:
on_faults.dtypes

RecordID                              int64
ESS_Id                                int64
EventTimeStamp               datetime64[ns]
eventDescription                     object
ecuSoftwareVersion                   object
ecuSerialNumber                      object
ecuModel                             object
ecuMake                              object
ecuSource                             int64
spn                                   int64
fmi                                   int64
active                                 bool
activeTransitionCount                 int64
EquipmentID                          object
MCTNumber                             int64
Latitude                            float64
Longitude                           float64
LocationTimeStamp            datetime64[ns]
EventDate                    datetime64[ns]
EventTime                            object
LocationDate                 datetime64[ns]
LocationTime                         object
AcceleratorPedal                

So we need all readouts of codes other than the breakdown down within a time period of 1 week before the breakdown code occured. 

WE will have to get some sort of aggregate of events occuring up to the derate and then make new binary features of weather or not they occured. Then our result is weather or not the truck derates.  

Each column of the df will be time periods:

7-6 days prior total # of non derate codes (ie not 5246)

6-5 '''

etc.

In [5]:
window_size = pd.Timedelta('5 hours')

In [6]:
df = on_faults.set_index('EventTimeStamp').sort_index(ascending = False)

Make dummy columns for category of spn, get_dummies function

In [8]:
df = (pd.get_dummies(data = df, columns =['spn'])
        .filter(regex='^(spn|EquipmentID)'))

In [9]:
df

,EquipmentID,spn_0,spn_16,spn_27,spn_33,spn_37,spn_38,spn_51,spn_70,spn_74,...,spn_520413,spn_520953,spn_521032,spn_523530,spn_523531,spn_523543,spn_524033,spn_524037,spn_524071,spn_524287
EventTimeStamp,,,,,,,,,,,,,,,,,,,,,
2020-03-06 14:15:34,2377,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:14:13,2377,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:13:38,1850,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:04:23,1994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-03-06 14:00:26,2282,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-02-21 11:18:20,310,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:18:20,310,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-02-21 11:14:38,309,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
agg_dict = {col: 'sum' for col in df.columns if col != 'EquipmentID'}

In [14]:
agg_dict['EquipmentID'] = lambda x : x[-1]

In [20]:
df.dtypes

EquipmentID    object
spn_0           uint8
spn_16          uint8
spn_27          uint8
spn_33          uint8
                ...  
spn_523543      uint8
spn_524033      uint8
spn_524037      uint8
spn_524071      uint8
spn_524287      uint8
Length: 420, dtype: object

In [25]:
(df
    # .set_index('EventTimeStamp')
    # .groupby(by = 'EquipmentID')
    .rolling(window = window_size)
    .agg(agg_dict)
    .reset_index()
    # 
            

)

DataError: No numeric types to aggregate